In [ ]:
# code by Rayn Lakha

In [ ]:
# importing useful libraries
%matplotlib ipympl
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib
from matplotlib import animation
from scipy import stats

In [ ]:
# Setting up environment
n = 6 # number of atoms
D = 2 # number of dimensions of system
T = 298 # temperature of system in Kelvin
Mr = 39.9 # relative atomic mass of Argon
sigma_lj = 3.4 # distance between two Argon atoms in Angstroms when their lj potential = 0. It = 2* van der Waals radius of an Argon atom in Angstroms
epsilon_lj = 0.997 # largest possible negative lj potential of two Argon atoms in kJ/mol
L = 3*n*sigma_lj # length of square unit cell in Angstroms (sufficiently large)
kB = 1.380649 * 10**(-23) # Boltzmann constant in J/K
pi = math.pi # defining the constant pi
m = 6.6335209 * 10**(-26) # mass of an Argon atom in kg
N_A = 6.022 * 10**23 # Avogadro's number in mol^(-1)
dt = 1 # timestep of simulation in tens of fs
atomic_area = pi * (sigma_lj/2)**2

In [ ]:
# Establishing initial positions of particles
init_positions = L*np.random.rand(n,D) # array of initial positions of n particles in D dimensional square unit cell with length L
print(init_positions) # in Angstrom coordinates

current_positions = init_positions #establishing current positions of the particles in Angstrom coordinates

In [ ]:
# defining the probability density function of the Maxwell-Boltzmann distribution. Units are s/metre.
# v is the speed of particle in metres/s, m is the mass of a particle in kg, and T is the temperature in K.
def MaxBol_pdf(v):
    T1 = 4*pi*(v**2) # term 1 of the pdf
    T2 = (m/(2*pi*kB*T))**(3/2) # term 2 of the pdf
    T3 = math.exp((-m)*(v**2)/(2*kB*T)) # term 3 of the pdf
    pdf = T1 * T2 * T3
    return pdf


In [ ]:
# defining a random sampling method from the Maxwell-Boltzmann pdf using acceptance-rejection sampling
# m is the mass of a particle in kg and T is the temperature of the system in K.
def MaxBol_sample():
    v_min = 0 # minimum possible velocity in m/s
    v_max = 1000 # maximum possible velocity in m/s
    upper_bound = 0.0025 # rough upper bound for pdf in s/m
    while True: # creating a loop
        v_rand = random.uniform(v_min, v_max) # selecting a random speed in chosen domain
        rand_pd = random.uniform(0, upper_bound) # selecting a random probability density in chosen range
        pd = MaxBol_pdf(v_rand) # finding actual probability density for our chosen speed
        if rand_pd < pd: # Does (v_rand, rand_pd) fall in the pdf?
            return v_rand # if so, return our random speed in m/s

In [ ]:
# Establishing initial speeds of particles
init_speeds = np.array(MaxBol_sample()) # set up the array of initial speeds with one element (one random speed in m/s)
for i in range(n-1): # add on n-1 more random inital speeds to the array so there are n randomly chosen speeds overall
    rand_speed = np.array(MaxBol_sample())
    init_speeds = np.vstack((init_speeds, rand_speed)) # in m/s
init_speeds = init_speeds * 10**(-4) # in A/(10fs)
print(init_speeds) # in A/(10fs)

In [ ]:
# Establishing initial velocities of particles
init_velsx = np.array(0) # set up the array of the x components of initial velocities with one element (0)
init_velsy = np.array(0) # set up the array of the x components of initial velocities with one element (0)
for speed in init_speeds: # for each of our random speeds find a corresponding random vx and vy
    rand_angle = 2*pi*np.random.random() # choose a random angle anticlockwise from the positive x direction in which the particle's velocity will act
    rand_velx = math.cos(rand_angle)*speed # find the x component of the random velocity
    init_velsx = np.vstack((init_velsx, rand_velx)) # attach this value to the array of the x components of initial velocities
    rand_vely = math.sin(rand_angle)*speed # find the y component of the random velocity
    init_velsy = np.vstack((init_velsy, rand_vely)) # attach this value to the array of the y components of initial velocities
init_velsx = init_velsx[1:, :] # remove the original 0 element
init_velsy = init_velsy[1:, :] # remove the original 0 element
init_vels = np.hstack((init_velsx, init_velsy)) # concatenate the x and y components of the random velocities to give an array of random velocities
print(init_vels) # in A/(10fs)

current_vels = init_vels # establishing current velocities of the particles in A/(10fs)

In [ ]:
# Defining lj potential energy calculation
def lj_U(d2): # d2 = the squared distance between two particles (d^2) (any units)
    T1 = (sigma_lj**12)/(d2**6)
    T2 = (sigma_lj**6)/(d2**3)
    potential = 4*epsilon_lj*(T1 - T2)
    return potential # in kJ/mol

In [ ]:
# Defining lj force magnitude calculation
def lj_F(d2): # d2 = the squared distance between two particles (d^2) in Angstroms^2
    T1 = (sigma_lj**12)/(d2**(13/2))
    T2 = 0.5 * (sigma_lj)**6/(d2**(7/2))
    force = 48*epsilon_lj*(T1-T2) # in kJ(mol^-1)(Angstroms^-1)
    force = force/(6.022 * 10**28) # convert to kgA(10fs)^(-2) per interaction
    return force 

In [ ]:
# Calculating current total potential energy
def U_calculator():
    U_total = 0
    for i in range(n-1): # for each particle i (except last one to avoid double-counting)
        for j in range(i+1, n): # for each non-already counted pair involving i
            dx = current_positions[j, 0] - current_positions[i, 0] # in Angstroms
            dx = dx - L*int(2*dx/L) # in Angstroms. Applying nearest image convention
            dy = current_positions[j, 1] - current_positions[i, 1] # in Angstroms
            dy = dy - L*int(2*dy/L) # in Angstroms. Applying nearest image convention
            distance2 = dx**2 + dy**2 # calculate squared distance in Angstroms^2 between i and j with PBCs applied.
            U_of_mole_of_interaction = lj_U(distance2) # in kJ/mol
            U_total = U_total + U_of_mole_of_interaction # in kJ/mol
    U_total = U_total * 10**18 / N_A # in femtoJoules
    return U_total # in femtoJoules

U_calculator()


In [ ]:
# Calculating current speeds
def speeds():
    all_speeds = np.array(0) # create an array to contain the speeds of the n particles
    for row_index in range(n): # for each particle
        sum_of_squares = 0 # set the sum of the squares of its perpendicular velocity components to 0
        for column_index in range(D): # for each perpendicular velocity component
            v_component2 = (current_vels[row_index, column_index])**2 # square that velocity component
            sum_of_squares = sum_of_squares + v_component2 # and add it to the sum of the particle's squared velocity components
        particle_speed = math.sqrt(sum_of_squares) # find the speed of the particle
        all_speeds = np.vstack((all_speeds, particle_speed)) # attach the particle's speed to the bottom of the all_speeds array
    all_speeds = all_speeds[1:, :] # remove the original 0 element to give the speeds of all particles
    return all_speeds # in A/(10fs)

current_speeds = speeds()
print(current_speeds)

In [ ]:
# Calculating prev. total kinetic energy
def ke_calculator():
    ke_total = 0
    for row_index in range(n): # for each particle
        particle_speed = current_speeds[row_index, 0] # in A(10fs)^(-1)
        particle_ke = 0.5*m * particle_speed**2 # in kgA^(2)(10fs)^(-2)
        ke_total = ke_total + particle_ke # in kgA^(2)(10fs)^(-2)
    ke_total = ke_total * 10**23 # convert to femtoJoules
    return ke_total # in femtoJoules
ke_calculator()

In [ ]:
# Calculating lj forces between particles with periodic boundary conditions (PBCs) applied
def forces():
    All_Fs = np.array([0, 0])
    for i in range(0, n): # for each particle i *starting from index 0* 
        resultant_force = np.array([0,0]) # set resultant force to 0 for each new particle i
        for j in [x for x in range(n) if x != i]: # for each other particle j
            dx = current_positions[j, 0] - current_positions[i, 0] # in Angstroms
            dx = dx - L*int(2*dx/L) # in Angstroms. Applying nearest image convention.
            dy = current_positions[j, 1] - current_positions[i, 1] # in Angstroms
            dy = dy - L*int(2*dy/L) # in Angstroms. Applying nearest image convention
            distance2 = dx**2 + dy**2 # calculate squared distance in Angstroms^2 between i and j with PBCs applied.

            displacement_ij = np.subtract(current_positions[j, :], current_positions[i, :]) # in Angstroms
            unit_vector = displacement_ij/math.sqrt(distance2) # divide displacement vector by its magnitude to give unit vector (magnitude of 1 A) in direction of i to j

            force = lj_F(distance2)*unit_vector # multiply magnitude of force by its direction to give force vector
            resultant_force = resultant_force + force # in kgA(10fs)^(-2) per interaction
        
        All_Fs = np.vstack((All_Fs, resultant_force))
    
    All_Fs = All_Fs[1:, :] # in kgA(10fs)^(-2) per interaction. Remove original 0 elements
    return All_Fs

init_Frces = forces() # in kgA(10fs)^(-2)
print(init_Frces)
init_accs = init_Frces / m # in A(10fs)^(-2)
print(init_accs)


In [ ]:
# Simulating particle movement using Stormer-Verlet method
sum_U = 0
sum_ke = 0
sum_U_contribution = 0

# Starting iteration
current_forces = forces() # find forces at t=0
current_as = current_forces/m # find as at t=0
prev_positions = current_positions # declare prev_positions = positions at t=0
future_positions = current_positions + current_vels*dt + 0.5*current_as*(dt**2) # find positions at t = dt. We use ds = Ut + 0.5at^2 (Euler)
current_positions = future_positions # advance particles: declare current_positions =  positions at t = dt

# Keeping particles in the unit cell
for row_index in range(n): # for each row
    for column_index in range(D): # for each column
        current_positions[row_index, column_index] = current_positions[row_index, column_index] - L* np.floor(current_positions[row_index, column_index] / L) # shift the particle back into the unit cell


# current now refers to t = T. Previous describes t = T - dt. Future describes t = T + dt
# we start the following loop with T = dt

def advance(run): # A function to advance the particles' positions and velocities. At the start of each loop / function call we say T = t.
    global sum_U, sum_ke, sum_U_contribution, current_positions, prev_positions, future_positions, current_forces, current_as, current_vels, current_speeds

    # advance forces and accelerations to catch up with the positions we just advanced.
    current_forces = forces() # in kgA(10fs)^(-2). Find forces at t = T
    current_as = current_forces/m # Use Newton's second law: a = F/m to give accelerations in A(10fs)^(-2) at t = T

    current_U = U_calculator() # find U at t = T
    future_positions = 2*current_positions - prev_positions + current_as * dt**2 # find positions in Angstrom coordinates at t = T + dt. We use the basic Verlet algorithm
    current_vels = (future_positions - prev_positions)/(2*dt) # find velocities in A/(10fs) at t = T (one timestep behind positions at T + dt). We use the Stormer-Verlet method
    current_speeds = speeds() # in A/(10fs) at t = T
    current_ke = ke_calculator() # find total ke at t = T
    current_energy = current_ke + current_U # find total energy at t = T in femtoJoules. Total energy should remain constant as time advances and this loop repeats many times (conservation of energy).

    # finding averages
    sum_U += current_U
    av_U = sum_U / run
    sum_ke += current_ke
    av_ke = sum_ke / run

    U_contribution = current_U / current_energy
    sum_U_contribution += U_contribution
    av_U_contribution = 100 * sum_U_contribution / run # in %
    av_ke_contribution = 100 - av_U_contribution # in %. Since U is usually negative, this is usually > 100%.
    #print(av_ke_contribution)

    # advance positions
    prev_positions = current_positions # declare previous positions = positions at t = T
    current_positions = future_positions # declare current_positions = positions at t = T + dt

    # Keeping particles in the unit cell
    for row_index in range(n): # for each row
        for column_index in range(D): # for each column
            current_positions[row_index, column_index] = current_positions[row_index, column_index] - L* np.floor(current_positions[row_index, column_index] / L) # shift the particle back into the unit cell
    
    return(prev_positions, current_energy) # returns positions and total energy at t = T

In [ ]:
# Visualising the system's energy over time and particles' movements
plt.ion() # initiate matplotlib's interactive mode
num_iterations = 600
iterations_list = range(1, num_iterations + 1) # number of iterations
total_energies = []
times_list = []

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6)) # creating a figure with two subplots

ax1.set(xlabel='time, tens of fs', ylabel='energy of system in femtoJoules')
ax1.set_title('Energy of system vs time', fontweight = 'bold')
ax1.set_xlim([0, num_iterations])
ax1.set_ylim([0, 2*10**(-4)])
line, = ax1.plot(times_list, total_energies) # one subplot shows changes in the energy of the system over time

ax2.set_xlim([0, L])
ax2.set_ylim([0, L])
ax2.set(xlabel='Angstroms', ylabel='Angstroms')
ax2.set_title('Argon atoms in unit cell', fontweight = 'bold')
particle_visualisation = ax2.scatter([], [], s = atomic_area) # the other subplot shows the positions of the particles' in the unit cell

# We start the following loop with T = dt
def visualise(iteration):
    global times_list, total_energies
    time_passed = iteration*dt # T = iteration * dt in 10s of fs

    positions_at_T, total_energy_at_T = advance(iteration) # gives the particles' positions and the systems' total energy at t = T
    particle_visualisation.set_offsets(positions_at_T) # move the particles on the particle trajectory visualisation subplot

    times_list.append(time_passed) # append the current time to the list of times
    total_energies.append(total_energy_at_T) # append the current energy to the list of energies
    line.set_data(times_list, total_energies) # draw a line of energy against time using these two lists

    
    return(line, particle_visualisation)

anim = matplotlib.animation.FuncAnimation(fig=fig, func=visualise, frames=iterations_list, interval=100, save_count = num_iterations, repeat = False)
# This function calls the visualise function (and hence the advance function) num_iterations times, repeatedly advancing the particles' positions by dt.
# It animates the resulting motion and plots a graph of the system's energy over time
plt.show()